In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import numpy as np
import os
from utils import *
from gan import *

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.set_num_threads(2)

In [2]:
# load data
batch_size = 64

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.LSUN('/home/dayun/lsun', classes=['bedroom_train'], transform=transform)
train_loader = data.DataLoader(train_data, batch_size=batch_size, num_workers = 4, shuffle=True)


# MLP-4

In [4]:
generator_iters = 600000
n_critic = 5

lr = 0.00005
c = 0.01

device = 'cuda' if torch.cuda.is_available() else 'cpu'

generator = MLP_G()
discriminator = DCGAN_D()

generator.apply(weights_init)
discriminator.apply(weights_init)

DCGAN_D(
  (conv): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

In [5]:
## train
optim_D = optim.RMSprop(discriminator.parameters(), lr=lr)
optim_G = optim.RMSprop(generator.parameters(), lr=lr)

loss_D = torch.cuda.FloatTensor if device == 'cuda' else torch.FloatTensor
loss_G = torch.cuda.FloatTensor if device == 'cuda' else torch.FloatTensor

data = get_infinite_batches(train_loader)
LOSS = {'D_real': [],
        'D_fake': [],
        'G': [],
        'W': []}

if not os.path.exists('result_MLP/'):
    os.makedirs('result_MLP/')
    
if device == 'cuda':
    generator.to(device)
    discriminator.to(device)
    cudnn.benchmark = True        
    
for g in range(generator_iters+1):
    # discriminator
    for p in discriminator.parameters():
        p.requires_grad = True
        
    for n in range(n_critic):    
        discriminator.zero_grad()
        
        real = data.__next__().to(device).requires_grad_(True)
        D_loss_real = torch.mean(discriminator(real))
        
        z = torch.randn(batch_size, 100, 1, 1, device = device, requires_grad = True)
        fake = generator(z).reshape((64, 3, 64, 64))
        D_loss_fake = torch.mean(discriminator(fake))

        W_loss = -D_loss_real + D_loss_fake
        W_loss.backward()
        optim_D.step()
   
        # clip
        for p in discriminator.parameters():
            p.data.clamp_(-c, c)
        
    # generator
    for p in discriminator.parameters():
        p.requires_grad = False  # to avoid computation
        
    generator.zero_grad()
    
    z = torch.randn(batch_size, 100, 1, 1, device = device, requires_grad = True)
    fake = generator(z).reshape((64, 3, 64, 64))
    G_loss = -torch.mean(discriminator(fake))
    G_loss.backward()
    optim_G.step()
    
    
    LOSS['D_real'].append(D_loss_real.data.cpu().numpy())
    LOSS['D_fake'].append(D_loss_fake.data.cpu().numpy())
    LOSS['G'].append(G_loss.data.cpu().numpy())
    LOSS['W'].append(-W_loss.data.cpu().numpy())
            
    if g % 1000 == 0:
        # save model
        save_model(generator, discriminator, 'MLP')
        save_dict(LOSS, 'loss_MLP')
        
        # save generator image
        z = Variable(torch.randn(64, 100, 1, 1)).cuda()
        samples = generator(z).reshape((64, 3, 64, 64))
        samples = samples.mul(0.5).add(0.5)  # denormalize
        samples = samples.data.cpu()
        grid = torchvision.utils.make_grid(samples)
        torchvision.utils.save_image(grid, 'result_MLP/iter_{}.png'.format(str(g)))
       
        # print wasserstein distance
        print("epoch: %d/%d\tW loss:%.5f" %(g, generator_iters, -W_loss))


Models save to ./generator.pkl & ./discriminator.pkl
epoch: 0/600000	W loss:0.14695
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 1000/600000	W loss:3.07927
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 2000/600000	W loss:3.08658
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 3000/600000	W loss:3.03864
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 4000/600000	W loss:3.01968
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 5000/600000	W loss:3.03148
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 6000/600000	W loss:3.02610
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 7000/600000	W loss:3.01905
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 8000/600000	W loss:3.02458
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 9000/600000	W loss:3.04057
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 10000/600000	W loss:3.02728
Models save to ./generator.pkl & ./discrimina

epoch: 93000/600000	W loss:3.11413
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 94000/600000	W loss:3.11197
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 95000/600000	W loss:3.11581
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 96000/600000	W loss:3.11782
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 97000/600000	W loss:3.11376
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 98000/600000	W loss:3.11378
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 99000/600000	W loss:3.09512
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 100000/600000	W loss:3.11878
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 101000/600000	W loss:3.11345
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 102000/600000	W loss:3.11738
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 103000/600000	W loss:3.11453
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 104000/600000	W loss:

Models save to ./generator.pkl & ./discriminator.pkl
epoch: 186000/600000	W loss:3.07067
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 187000/600000	W loss:3.11420
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 188000/600000	W loss:3.09762
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 189000/600000	W loss:3.04260
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 190000/600000	W loss:3.09722
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 191000/600000	W loss:3.11773
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 192000/600000	W loss:3.11336
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 193000/600000	W loss:3.04021
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 194000/600000	W loss:3.10748
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 195000/600000	W loss:3.06714
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 196000/600000	W loss:3.11260
Models save to ./gene

epoch: 278000/600000	W loss:3.11289
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 279000/600000	W loss:3.02381
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 280000/600000	W loss:3.06311
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 281000/600000	W loss:3.10260
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 282000/600000	W loss:3.00825
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 283000/600000	W loss:3.06509
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 284000/600000	W loss:3.07980
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 285000/600000	W loss:3.09608
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 286000/600000	W loss:3.09108
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 287000/600000	W loss:3.10796
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 288000/600000	W loss:3.08473
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 289000/600000	

Models save to ./generator.pkl & ./discriminator.pkl
epoch: 371000/600000	W loss:3.10165
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 372000/600000	W loss:3.10011
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 373000/600000	W loss:3.09580
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 374000/600000	W loss:2.07854
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 375000/600000	W loss:3.03249
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 376000/600000	W loss:2.29885
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 377000/600000	W loss:3.00940
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 378000/600000	W loss:3.09042
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 379000/600000	W loss:3.03244
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 380000/600000	W loss:3.09509
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 381000/600000	W loss:3.09363
Models save to ./gene

epoch: 463000/600000	W loss:3.08143
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 464000/600000	W loss:3.05366
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 465000/600000	W loss:3.07153
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 466000/600000	W loss:3.10295
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 467000/600000	W loss:2.98204
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 468000/600000	W loss:3.09839
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 469000/600000	W loss:3.09284
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 470000/600000	W loss:3.00222
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 471000/600000	W loss:3.08588
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 472000/600000	W loss:3.07565
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 473000/600000	W loss:3.04882
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 474000/600000	

Models save to ./generator.pkl & ./discriminator.pkl
epoch: 556000/600000	W loss:2.99874
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 557000/600000	W loss:3.09156
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 558000/600000	W loss:3.04047
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 559000/600000	W loss:3.07274
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 560000/600000	W loss:3.05202
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 561000/600000	W loss:3.01340
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 562000/600000	W loss:3.06239
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 563000/600000	W loss:3.05871
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 564000/600000	W loss:3.01301
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 565000/600000	W loss:3.08191
Models save to ./generator.pkl & ./discriminator.pkl
epoch: 566000/600000	W loss:3.09069
Models save to ./gene

# MPT-5

In [5]:
generator_iters = 600000
n_critic = 5
alpha = 0.00005
c = 0.01

device = 'cuda' if torch.cuda.is_available() else 'cpu'

generator = MLP_G2()
discriminator = DCGAN_D()

generator.apply(weights_init)
discriminator.apply(weights_init)

DCGAN_D(
  (conv): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

In [6]:
## train
optim_D = optim.RMSprop(discriminator.parameters(), lr=alpha)
optim_G = optim.RMSprop(generator.parameters(), lr=alpha)

loss_D = torch.cuda.FloatTensor if device == 'cuda' else torch.FloatTensor
loss_G = torch.cuda.FloatTensor if device == 'cuda' else torch.FloatTensor

data = get_infinite_batches(train_loader)
LOSS = {'D_real': [],
        'D_fake': [],
        'G': [],
        'W': []}

if not os.path.exists('result_MLP2/'):
    os.makedirs('result_MLP2/')
    
if device == 'cuda':
    generator.to(device)
    discriminator.to(device)
    cudnn.benchmark = True        
    
for g in range(generator_iters+1):
    # discriminator
    for p in discriminator.parameters():
        p.requires_grad = True
        
    for n in range(n_critic):    
        discriminator.zero_grad()
        
        real = data.__next__().to(device).requires_grad_(True)
        D_loss_real = torch.mean(discriminator(real))
        
        z = torch.randn(batch_size, 100, 1, 1, device = device, requires_grad = True)
        fake = generator(z).reshape((64, 3, 64, 64))
        D_loss_fake = torch.mean(discriminator(fake))

        W_loss = -D_loss_real + D_loss_fake
        W_loss.backward()
        optim_D.step()
   
        # clip
        for p in discriminator.parameters():
            p.data.clamp_(-c, c)
        
    # generator
    for p in discriminator.parameters():
        p.requires_grad = False  # to avoid computation
        
    generator.zero_grad()
    
    z = torch.randn(batch_size, 100, 1, 1, device = device, requires_grad = True)
    fake = generator(z).reshape((64, 3, 64, 64))
    G_loss = -torch.mean(discriminator(fake))
    G_loss.backward()
    optim_G.step()
    
    
    LOSS['D_real'].append(D_loss_real.data.cpu().numpy())
    LOSS['D_fake'].append(D_loss_fake.data.cpu().numpy())
    LOSS['G'].append(G_loss.data.cpu().numpy())
    LOSS['W'].append(-W_loss.data.cpu().numpy())
            
    if g % 1000 == 0:
        # save model
        save_model(generator, discriminator, 'MLP2')
        save_dict(LOSS, 'loss_MLP2')
        
        # save generator image
        z = Variable(torch.randn(64, 100, 1, 1)).cuda()
        samples = generator(z).reshape((64, 3, 64, 64))
        samples = samples.mul(0.5).add(0.5)  # denormalize
        samples = samples.data.cpu()
        grid = torchvision.utils.make_grid(samples)
        torchvision.utils.save_image(grid, 'result_MLP2/iter_{}.png'.format(str(g)))
       
        # print wasserstein distance
        print("epoch: %d/%d\tW loss:%.5f" %(g, generator_iters, -W_loss))


epoch: 0/600000	W loss:0.14571
epoch: 1000/600000	W loss:3.06504
epoch: 2000/600000	W loss:3.00207
epoch: 3000/600000	W loss:3.01684
epoch: 4000/600000	W loss:3.04005
epoch: 5000/600000	W loss:2.99988
epoch: 6000/600000	W loss:2.99469
epoch: 7000/600000	W loss:3.01356
epoch: 8000/600000	W loss:3.01378
epoch: 9000/600000	W loss:3.02753
epoch: 10000/600000	W loss:3.02897
epoch: 11000/600000	W loss:3.01373
epoch: 12000/600000	W loss:3.02453
epoch: 13000/600000	W loss:3.03429
epoch: 14000/600000	W loss:3.03484
epoch: 15000/600000	W loss:3.04006
epoch: 16000/600000	W loss:3.02483
epoch: 17000/600000	W loss:3.02687
epoch: 18000/600000	W loss:3.02130
epoch: 19000/600000	W loss:3.01965
epoch: 20000/600000	W loss:3.01948
epoch: 21000/600000	W loss:3.00412
epoch: 22000/600000	W loss:3.02134
epoch: 23000/600000	W loss:3.03949
epoch: 24000/600000	W loss:3.03503
epoch: 25000/600000	W loss:2.89867
epoch: 26000/600000	W loss:3.01270
epoch: 27000/600000	W loss:3.01470
epoch: 28000/600000	W loss:3.0014

epoch: 231000/600000	W loss:1.99498
epoch: 232000/600000	W loss:1.82052
epoch: 233000/600000	W loss:1.93680
epoch: 234000/600000	W loss:2.02747
epoch: 235000/600000	W loss:1.96032
epoch: 236000/600000	W loss:1.85547
epoch: 237000/600000	W loss:1.93221
epoch: 238000/600000	W loss:1.72000
epoch: 239000/600000	W loss:1.89921
epoch: 240000/600000	W loss:1.71244
epoch: 241000/600000	W loss:1.96055
epoch: 242000/600000	W loss:1.81785
epoch: 243000/600000	W loss:1.97104
epoch: 244000/600000	W loss:1.95418
epoch: 245000/600000	W loss:1.93687
epoch: 246000/600000	W loss:1.92912
epoch: 247000/600000	W loss:1.80144
epoch: 248000/600000	W loss:1.88380
epoch: 249000/600000	W loss:1.89651
epoch: 250000/600000	W loss:1.91940
epoch: 251000/600000	W loss:1.84996
epoch: 252000/600000	W loss:1.92849
epoch: 253000/600000	W loss:1.92523
epoch: 254000/600000	W loss:1.87165
epoch: 255000/600000	W loss:1.93602
epoch: 256000/600000	W loss:1.69560
epoch: 257000/600000	W loss:1.98050
epoch: 258000/600000	W loss:

epoch: 459000/600000	W loss:1.39637
epoch: 460000/600000	W loss:1.51641
epoch: 461000/600000	W loss:1.44608
epoch: 462000/600000	W loss:1.49362
epoch: 463000/600000	W loss:1.51454
epoch: 464000/600000	W loss:1.45277
epoch: 465000/600000	W loss:1.54468
epoch: 466000/600000	W loss:1.54692
epoch: 467000/600000	W loss:1.48539
epoch: 468000/600000	W loss:1.45866
epoch: 469000/600000	W loss:1.32179
epoch: 470000/600000	W loss:1.34911
epoch: 471000/600000	W loss:1.56507
epoch: 472000/600000	W loss:1.31039
epoch: 473000/600000	W loss:1.45650
epoch: 474000/600000	W loss:1.53259
epoch: 475000/600000	W loss:1.37409
epoch: 476000/600000	W loss:1.60251
epoch: 477000/600000	W loss:1.54243
epoch: 478000/600000	W loss:1.38698
epoch: 479000/600000	W loss:1.57492
epoch: 480000/600000	W loss:1.51219
epoch: 481000/600000	W loss:1.48674
epoch: 482000/600000	W loss:1.46852
epoch: 483000/600000	W loss:1.53119
epoch: 484000/600000	W loss:1.32617
epoch: 485000/600000	W loss:1.35222
epoch: 486000/600000	W loss: